In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from scipy.linalg import eig, inv
import statsmodels.api as sm
import pandas as pd
import numpy as np


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.precision', 3)

In [3]:
df = fetch_california_housing(as_frame=True).frame
data_columns = list(df.columns)[:-1]
print(df.columns)
df_standarized = df.copy()
df_standarized[list(df.columns)] = StandardScaler().fit_transform(df[list(df.columns)])
X = np.asmatrix(df_standarized[data_columns].values)
A = X.T * X
lambdas, T = eig(A)
T = T.T
# np.asmatrix(v).T * X * np.asmatrix(v) # == lambdas (look at the diagonal: they are the same)
lambda_max = max(lambdas)
print(lambda_max)

Index(['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup',
       'Latitude', 'Longitude', 'MedHouseVal'],
      dtype='object')
(41836.236095122265+0j)


In [4]:
Z = X * T
Y = np.asmatrix(df.MedHouseVal.values).T
Y, Y.shape
# alpha_OLS = inv(Z.T * Z)*Z.T * Y.T
# alpha_OLS

(matrix([[4.526],
         [3.585],
         [3.521],
         ...,
         [0.923],
         [0.847],
         [0.894]]),
 (20640, 1))

In [5]:
# df_standarized["constant"] = sm.add_constant(np.ones(df.MedHouseVal.shape))
model = sm.OLS(df_standarized.MedHouseVal,df_standarized[data_columns])
results = model.fit()
beta_OLS = np.asmatrix(results.params).T
# Y_prime = X * beta_OLS.T
# print(Y_prime)
beta_OLS, beta_OLS.shape

(matrix([[ 0.71895227],
         [ 0.10291078],
         [-0.23010693],
         [ 0.26491789],
         [-0.00390232],
         [-0.03408034],
         [-0.77984545],
         [-0.75441522]]),
 (8, 1))

In [6]:
alpha_OLS = (inv(T) * beta_OLS)
n, p = df.shape[0], len(data_columns)
std_err = (Y.T*Y - (alpha_OLS.T * Z.T) * Y)/(n-p-1)
alpha_OLS, std_err

(matrix([[-0.73823032],
         [-0.29371963],
         [-0.35062293],
         [ 0.04963813],
         [ 0.31767058],
         [-0.72227761],
         [-0.63012187],
         [ 0.22974528]]),
 matrix([[4.91307654]]))

$k_1$

In [7]:
print(p*std_err)
k_1=p*std_err/(alpha_OLS.T*alpha_OLS)
k_1

[[39.30461228]]


matrix([[21.48859661]])

$k_i(AD)$

In [8]:
k_iAD = 2*std_err/(lambda_max*alpha_OLS.A1**2)
np.real(k_iAD)

matrix([[0.00043097, 0.00272248, 0.00191051, 0.09532354, 0.00232743,
         0.00045022, 0.00059154, 0.00444977]])

$k_1(AD)$

In [9]:
k_1AD = 2/(8*lambda_max) * np.sum(std_err/alpha_OLS.A1**2 )
np.real(k_1AD)

0.013525807768928226

$k_2(AD)$

In [10]:
k_2AD = np.median([2*std_err/(lambda_max*alpha_OLS.A1**2)])
np.real(k_2AD)

0.0021189729385916543

$k_3(AD)$

In [17]:
k_3AD = 2*std_err/(lambda_max*np.prod([x**2 for x in alpha_OLS])**(1/p))
np.real(k_3AD.A1[0])

0.002219717496192362

$k_4(AD)$

In [12]:
k_4AD = (2*p/lambda_max) * sum([std_err/x**2 for x in alpha_OLS])
np.real(k_4AD).A1[0] 

0.8656516972114062

# Sources
- New ridge parameters for ridge regression